In [1]:
import wooldridge as woo
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats


# Functions that we build in previous chapters are saved in supplementary Functions
# Sometimes we will reconstruct and augment a past function. We will save the
#  more advanced functions in this file
from supplementaryFunctions import *

# 4. Multiple Regression Analysis: Inference

## 4.1. The _t_ Test

### 4.1.1. General Setup

$$H_0:\beta_j=a_j$$

$$H_1: \beta_j \neq a_j $$

#### One-tailed test

<center>$H_1: \beta_j < a_j $ or $ H1: \beta_j > a_j $</center>

#### _t_ statistic

$$t=\frac{\hat{\beta_j} - a_j}{se(\hat{\beta_j})}$$

### 4.1.2. Standard Case

$H_0: \beta_j = 0$, $H_1:\beta_j \neq 0$

$t_{\hat{\beta_j}} = \frac{\hat{\beta_j}}{se(\hat{\beta_j})}$

The subscript on the _t_ statistic indicates that this is "the" _t_ value for $\hat{\beta_j}$ for this frequent version of the test. Under $H_0$, it has the _t_ distribution with $n - k - 1$ degress of freedom implying that the probability that $|t_{\hat{\beta_j}}| > c$ is equal to $\alpha$ if $c$ is the $1 - \frac{\alpha}{2}$ quantile of this distribution. If $\alpha$ is our significance level (e.g., $\alpha = 5\%$), then we

<center>reject $H_0$ if $|t_{\hat{\beta_j}}| > c$</center>

in our sample. For the typical significance level $\alpha=5\%$, the critical value $c$ will be around 2 for reasonably large degrees of freedom and approach the counterpart of $1.96$ from the standard normal distribution in very large samples.

The _p_ value indicates the smallest value of the significance level $\alpha$ for which we would still reject $H_0$ using our sample. So it is the probability for a random variable $T$ with the respective $t$ distribution that $|T| >|t_{\hat{\beta_j}}|$ where $t_{\hat{\beta_j}}$ is the value of the $t$ statistic in our particular sample. In our two-tailed test, it can be calculated as

$$p_{\hat{\beta_j}}=2F_{t_{n-k-1}}(-|t_{\hat{\beta_j}}|)$$

where $F_{t_{n-k-1}}()$ is the CDF of the t distribution with $n-k-1$ degrees of freedom.

<center>reject $H_0$ if $p_{\hat{\beta_j}}\leq \alpha$</center>

### Example 4.3: Determinants of College GPA

In [2]:
gpa1 = woo.dataWoo("gpa1")

alpha = np.array([.05, .01, .001])
n = len(gpa1.index)
cv_t = stats.t.ppf(1- alpha / 2, n)
cv_t

array([1.97693149, 2.61114718, 3.36086555])

In [3]:
cv_norm = stats.norm.ppf(1 - alpha / 2)
cv_norm

array([1.95996398, 2.5758293 , 3.29052673])

In [4]:
names = ["colGPA", "hsGPA", "ACT", "skipped"]
data = gpa1[names]

X, y = build_X_y_matrices(data, names)

reg = sm.OLS(y, X)
results = reg.fit()
OLS_summary(results)

,$\beta $,$t$,$$P>|t|$$,$SE$
$$r^2: 0.2336$$,,,,
hsGPA,0.4118,4.3963,0.0000,0.0937
ACT,0.0147,1.3933,0.1658,0.0106
skipped,-0.0831,-3.1968,0.0017,0.0260
Intercept,1.3896,4.1910,0.0000,0.3316


In [5]:
# Reproduce t-stats
t = results.params / results.bse
t

hsGPA        4.396260
ACT          1.393319
skipped     -3.196840
Intercept    4.191039
dtype: float64

In [6]:
pval = pd.DataFrame(2 * stats.t.cdf(-abs(t), n), index = t.index).rename(columns = {0:"$p$"})
pval.round(2)

,$p$
hsGPA,0.00
ACT,0.17
skipped,0.00
Intercept,0.00


### 4.1.3. Other Hypotheses

|$$H_1$$|$\beta_j \neq \alpha_j$|$\beta_j > \alpha_j$|$\beta_j < \alpha_j$|
|---|---|---|---|
|$$c=:$$|$$1-\frac{\alpha}{2}$$|$$1-\alpha$$|$$1-\alpha$$|
|reject $H_0$ if$:$|$|\hat{\beta_j}|$>c|$\hat{\beta_j}>c$|$\hat{\beta_j}<c$|
|$p$ value$:$|$2F_{t_{n-k-1}}(-|t_{\hat{\beta_j}}|)$|$F_{t_{n-k-1}}(-t_{\hat{\beta_j}})$|$F_{t_{n-k-1}}(t_{\hat{\beta_j}})$|

__<center>Table 4.1. One- and Two-tailed Tests for $H_0: \beta_j=\alpha_j$</center>__

### Example 4.1: Hourly Wage Equation

In [7]:
wage1 = woo.dataWoo("wage1")
n = wage1.shape[0]

alpha = np.array([0.05, 0.01])
cv_t = stats.t.ppf(1- alpha, n)
cv_t

array([1.64775567, 2.33345809])

In [8]:
cv_norm = stats.norm.ppf(1 - alpha)
cv_norm

array([1.64485363, 2.32634787])

In [9]:
names = ["wage", "educ", "exper", "tenure"]
log_vars = ["wage"]
data = wage1[names]
X, y = build_X_y_matrices(data, names, log_vars)

reg = sm.OLS(y,X)
results = reg.fit()

OLS_summary(results)

C:\Users\JLCat\GithubRepositories\OtherRepositories\Wooldridge\supplementaryFunctions.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[name] = np.log(data[name])


,$\beta $,$t$,$$P>|t|$$,$SE$
$$r^2: 0.316$$,,,,
educ,0.0920,12.5552,0.0000,0.0073
exper,0.0041,2.3914,0.0171,0.0017
tenure,0.0221,7.1331,0.0000,0.0031
Intercept,0.2844,2.7292,0.0066,0.1042


### 4.2 Confidence Intervals

$$\hat{\beta_j} \mp c*se(\hat{\beta_j})$$

In [10]:
rdchem = woo.dataWoo("rdchem")
names = ["rd", "sales", "profmarg"]
log_vars = names[:2]
data = rdchem[names]
X, y = build_X_y_matrices(data, names, log_vars)

reg = sm.OLS(y, X)
results = reg.fit()

OLS_summary(results)

C:\Users\JLCat\GithubRepositories\OtherRepositories\Wooldridge\supplementaryFunctions.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[name] = np.log(data[name])


,$\beta $,$t$,$$P>|t|$$,$SE$
$$r^2: 0.918$$,,,,
sales,1.0842,18.0118,0.000,0.0602
profmarg,0.0217,1.6941,0.101,0.0128
Intercept,-4.3783,-9.3549,0.000,0.4680


In [11]:
ci_values = [.9, .95,.99, .999]
CI_dict = {}
for ci in ci_values:
    CI_dict[ci] = results.conf_int(1 - ci).rename(columns = {0:"Lower", 1:"Upper"})
CI_dict

{0.9:               Lower     Upper
 sales      0.981941  1.186499
 profmarg  -0.000064  0.043375
 Intercept -5.173496 -3.583050,
 0.95:               Lower     Upper
 sales      0.961107  1.207332
 profmarg  -0.004488  0.047799
 Intercept -5.335478 -3.421068,
 0.99:               Lower     Upper
 sales      0.918299  1.250141
 profmarg  -0.013578  0.056890
 Intercept -5.668313 -3.088234,
 0.999:               Lower     Upper
 sales      0.863942  1.304498
 profmarg  -0.025121  0.068433
 Intercept -6.090942 -2.665604}

In [12]:
for ci in ci_values:
    print("CI: " + str(ci) ,CI_dict[ci], sep = "\n")

CI: 0.9
              Lower     Upper
sales      0.981941  1.186499
profmarg  -0.000064  0.043375
Intercept -5.173496 -3.583050
CI: 0.95
              Lower     Upper
sales      0.961107  1.207332
profmarg  -0.004488  0.047799
Intercept -5.335478 -3.421068
CI: 0.99
              Lower     Upper
sales      0.918299  1.250141
profmarg  -0.013578  0.056890
Intercept -5.668313 -3.088234
CI: 0.999
              Lower     Upper
sales      0.863942  1.304498
profmarg  -0.025121  0.068433
Intercept -6.090942 -2.665604


## 4.3. Linear Restrictions: _F_ Tests

The f statistic mesaures the average reduction in unexplained variance for each additional explanatory variable added to an equation: 

$$ F = \frac{SSR_r - SSR_{ur}}{SSR_{ur}} \frac{n-k-1}{q} = \frac{R_{ur}^2 - R_r^2}{1 - R_{ur}^2} \frac{n - k - 1}{q} $$

In [13]:
mlb1 = woo.dataWoo("mlb1")
n = mlb1.shape[0]
F_test_stats = {}


#ur regression
names = ["salary", "years", "gamesyr", "bavg", "hrunsyr", "rbisyr"]
log_vars = ["salary"]
data = mlb1[names]
X, y = build_X_y_matrices(data, names, log_vars)
k = len(names) - 1
reg_ur = sm.OLS(y, X)
results_ur = reg_ur.fit()
OLS_summary(results_ur)

C:\Users\JLCat\GithubRepositories\OtherRepositories\Wooldridge\supplementaryFunctions.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[name] = np.log(data[name])


,$\beta $,$t$,$$P>|t|$$,$SE$
$$r^2: 0.6278$$,,,,
years,0.0689,5.6843,0.0000,0.0121
gamesyr,0.0126,4.7424,0.0000,0.0026
bavg,0.0010,0.8868,0.3758,0.0011
hrunsyr,0.0144,0.8986,0.3695,0.0161
rbisyr,0.0108,1.5005,0.1344,0.0072
Intercept,11.1924,38.7518,0.0000,0.2888


In [14]:
r2_ur = results_ur.rsquared
F_test_stats["r2_ur"] = r2_ur

#r regression
names = ["salary", "years", "gamesyr"]
data = mlb1[names]
q = len(names)
X, y = build_X_y_matrices(data, names, log_vars)

reg_r = sm.OLS(y, X)
results_r = reg_r.fit()
OLS_summary(results_r)

C:\Users\JLCat\GithubRepositories\OtherRepositories\Wooldridge\supplementaryFunctions.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[name] = np.log(data[name])


,$\beta $,$t$,$$P>|t|$$,$SE$
$$r^2: 0.5971$$,,,,
years,0.0713,5.7032,0.0,0.0125
gamesyr,0.0202,15.0234,0.0,0.0013
Intercept,11.2238,103.6247,0.0,0.1083


In [15]:
r2_r = results_r.rsquared
F_test_stats["r2_r"] = r2_r

# F Statistic
cv = stats.f.ppf(1 - 0.01, len(X.keys()), n - k - 1)
fstat = (r2_ur - r2_r)/(1 - r2_ur) * (n - k - 1)/q
F_test_stats["F-stat (Formula)"] = fstat
fpval = 1 - stats.f.cdf(fstat,q, n - k - 1)
F_test_stats["p-val"] = fpval
F_test_stats["F-stat (scipy)"] = stats.f.ppf(1 - fpval, q, n - k - 1)

print("cv p = 0.01:", cv)

pd.DataFrame(F_test_stats, index =["Value"]).T

cv p = 0.01: 3.838520048496057


,Value
r2_ur,0.627803
r2_r,0.597072
F-stat (Formula),9.550254
p-val,0.000004
F-stat (scipy),9.550254


Statsmodels has a great feature that allows us to explicitly write out our F-tests hypotheses. We will create a list of hypotheses:

> hypotheses = ["var_name1 = 0", "var_name2 = 0", . . .]

and we will run the ftest using results from the unrestricted regression:

> ftest = results.f_test(hypotheses)

In [16]:
names = ["salary", "years", "gamesyr", "bavg", "hrunsyr", "rbisyr"]
data = mlb1[names]
X, y = build_X_y_matrices(data, names, log_vars)

reg = sm.OLS(y, X)
results = reg.fit()

f_test_results = {}
hypotheses = [name + " = 0" for name in names[-3:]]
hypotheses

C:\Users\JLCat\GithubRepositories\OtherRepositories\Wooldridge\supplementaryFunctions.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[name] = np.log(data[name])


['bavg = 0', 'hrunsyr = 0', 'rbisyr = 0']

In [17]:
# F Test
key = str(hypotheses).replace("[","").replace("]", "")
f_test_results[key] = {}
ftest = results.f_test(hypotheses)
f_test_results[key]["F-stat"] = ftest.statistic
f_test_results[key]["P_F"]= ftest.pvalue
f_test_results

{"'bavg = 0', 'hrunsyr = 0', 'rbisyr = 0'": {'F-stat': 9.550253521952135,
  'P_F': 4.4737081398374674e-06}}

We could check multiple hypothese using a for loop. Let's check each variable individually.

In [18]:
for hypothesis in hypotheses:
    key = hypothesis
    hypothesis = [hypothesis]
    f_test_results[key] = {}
    ftest = results.f_test(hypothesis)
    f_test_results[key]["F-stat"] = ftest.statistic
    f_test_results[key]["P_F"]= ftest.pvalue
pd.DataFrame(f_test_results)

,"'bavg = 0', 'hrunsyr = 0', 'rbisyr = 0'",bavg = 0,hrunsyr = 0,rbisyr = 0
F-stat,9.550254,0.786433,0.807557,2.251377
P_F,0.000004,0.375795,0.369467,0.134405


It seems that individually the addition of these explanatory variables do not improve explanatory power of our regression. Let's see which pairs appear to improve explanation of variance. We will save the p-value of the F-statistic for each pair.

In [19]:
pair_hypotheses_results = {}
for hypothesis1 in hypotheses:
    pair_hypotheses_results[hypothesis1] = {}
    for hypothesis2 in hypotheses:
        if hypothesis1 != hypothesis2:
            hypothesis = [hypothesis1, hypothesis2]
            ftest = results.f_test(hypothesis)
            pair_hypotheses_results[hypothesis1][hypothesis2] = ftest.pvalue
pd.DataFrame(pair_hypotheses_results)

,bavg = 0,hrunsyr = 0,rbisyr = 0
hrunsyr = 0,0.526698,NaN,0.000002
rbisyr = 0,0.134924,0.000002,NaN
bavg = 0,NaN,0.526698,0.134924


It appears that only the inclusion of homeruns and RBIs improve the fit. In fact, the F-statistic generated for this pair $(p = 0.000002)$ is better than the value generated when bavg is add $(p = 0.000004)$

In [20]:
hypotheses = ["bavg = 0", "rbisyr = 4*hrunsyr"]
key = str(hypotheses).replace("[","").replace("]", "")
f_test_results[key] = {}
ftest = results.f_test(hypotheses)
f_test_results[key]["F-stat"] = ftest.statistic
f_test_results[key]["P_F"]= ftest.pvalue
pd.DataFrame(f_test_results)

,"'bavg = 0', 'hrunsyr = 0', 'rbisyr = 0'",bavg = 0,hrunsyr = 0,rbisyr = 0,"'bavg = 0', 'rbisyr = 4*hrunsyr'"
F-stat,9.550254,0.786433,0.807557,2.251377,0.499114
P_F,0.000004,0.375795,0.369467,0.134405,0.607504


### <center>Gauss-Markov Assumptions</center>

|Assumption No.|Brief Summary|Description|
|---|---|---|
|$MLR.1$| Linear in Parameters: $$y = \beta_0 + \beta_1x+u$$| In the population model, the dependent variable, $y$, is related to the independent variable, $x$, and the error term, $u$ where $\beta_0$ and $\beta_1$ are the population intercept and slope parameters, respectively. Assumption $MLR.1$ describes the population relationship we hope to estimate, and explicitly sets out the $\beta_j$ - the ceteris paribus population effects on the $x_j$ and $y$ 0 as the parameters of interest.|
|$MLR.2$| Random Sampling of $x$ and $y$ from the Population|We have a random sample of size $n$, ${(x_i,y_i): i = 1,2,...,n}$, following the population model in Assumption $MLR.1$. This random sampling assumption means that we have data that can be used to estimate the $\beta_j$, and that the data have been chosen to be representative of the population described in Assumption $MLR.1$|
|$MLR.3$| No Perfect Collinearity | In the sample (and therefore the population), none of the independent variables is constant, and there are no exact _linear_ relationships among the independent variables. Once we have a sample of data, we need to know that we can use the data to compute the OLS estimates, the $\hat{\beta_j}$. This is the role of Assumption $MLR.3$ if we have sample variation in each independent variable and no exact linear relationships among the independent variables, we can compute the $\hat{\beta_j}$.|
|$MLR.4$| Zero Conditional Mean: $E(u|x_1, x_2, . . . , x_k)=0$|The error $u$ has an expected value of zero given any values of the explanatory variables. Assuming that the unobserved factors are, on average, unrelated to the explanatory variables is key to deriving the first statistical property of each OLS estimator: its unbiasedness for the corresponding population parameter.|
|$MLR.5$| Homoscedasticity: $Var(u|x_1, x_2, . . . , x_k) = \sigma^2$|The error $u$ has constant variance given any value of the explanatory variables. Compared with Assumption $MLR.4$, the homoskedasticity assumption is of secondary importance; in particular, Assumption $MLR.5$ has no bearing on the unbiasedness of the $\hat{\beta_j}$ Still, homoskedasticity has two important implications: (1) We can derive formulas for the sampling variances whose components are easy to characterize; (2) We can conclusde, under the Gauss-Markov assumptions $MLR.1$ through $MLR.5$, that the OLS estimators have smallest variances among _all_ linear, unbiased estimators.|
|$MLR.6$| Normality: $u ~ Normal(0, \sigma^2)$|The population error $u$ is independent of the explanatory variables $x_1, x_2, . . . , x_k$) and is normally distributed withe zero mean and variabce $\sigma^2$. This assumption obtains the exact sampling distributions of t statistics and F statistics, so that we can carry out exact hypotheses tests. This assumption can be dropped if we have a reasonablly large sample size. This assumption does imply a stronger efficiency property of OLS: the OLS estimators have smallest variance among _all_ unbiased estimators; the comparison group is no longer restricted to estimators linear in the set $\{y_i:i=1,2,...,n\}$|